<a href="https://colab.research.google.com/github/Harshkotkar/Deep-Learning/blob/main/Early_Detection_of_Diabetic_Retinopathy_from_Retinal_Fundus_Images(full_and_final_%F0%9F%A4%AF%F0%9F%94%A5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aptos2019_blindness_detection_path = kagglehub.competition_download('aptos2019-blindness-detection')

print('Data source import complete.')


100%|██████████| 9.51G/9.51G [04:03<00:00, 41.9MB/s]

Extracting files...


Data source import complete.


In [5]:
 # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input



In [7]:
import os

print(aptos2019_blindness_detection_path)  # This shows the base folder
print(os.listdir(aptos2019_blindness_detection_path))  # List what’s inside


/root/.cache/kagglehub/competitions/aptos2019-blindness-detection
['train_images', 'train.csv', 'test.csv', 'test_images', 'sample_submission.csv']


In [8]:
import os
import pandas as pd

# Base dataset path
base_path = "/root/.cache/kagglehub/competitions/aptos2019-blindness-detection"

# Load CSVs
train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

print(train_df.head())
print(test_df.head())


        id_code  diagnosis
0  000c1434d8d7          2
1  001639a390f0          4
2  0024cdab0c1e          1
3  002c21358ce6          0
4  005b95c28852          0
        id_code
0  0005cfc8afb6
1  003f0afdcd15
2  006efc72b638
3  00836aaacf06
4  009245722fa4


In [9]:
base_path = "/root/.cache/kagglehub/competitions/aptos2019-blindness-detection"
train_imgs = os.path.join(base_path, "train_images")

In [10]:
train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

In [11]:
train_df["id_code"] = train_df["id_code"].str.replace(r"(\.png)+$", ".png", regex=True)
test_df["id_code"]  = test_df["id_code"].str.replace(r"(\.png)+$", ".png", regex=True)

In [13]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['diagnosis'], random_state=42)


In [14]:
# Ensure .png extension is appended correctly
train_df["id_code"] = train_df["id_code"].astype(str).str.replace(r"(\.png)+$", "", regex=True) + ".png"
val_df["id_code"]   = val_df["id_code"].astype(str).str.replace(r"(\.png)+$", "", regex=True) + ".png"

print(train_df.head())
print(val_df.head())

               id_code  diagnosis
2579  b2b79b37d314.png          0
309   175dd560810a.png          2
2826  c3cd0200df79.png          3
2989  cfed7c1172ec.png          0
3344  e811f39a1243.png          2
               id_code  diagnosis
1595  6fe67fd7f5d1.png          0
3449  ef8c39eb9157.png          0
1394  61bbc11fe503.png          4
1379  60edda7b4871.png          0
3043  d30d079e6f9a.png          0


In [15]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])


In [16]:
IMG_SIZE = 224
BATCH_SIZE = 32

def load_image(img_id, label, augment=False):
    img_path = tf.strings.join([train_imgs, "/", img_id])
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = preprocess_input(img)  # EfficientNet-specific preprocessing
    if augment:
        img = data_augmentation(img)
    return img, label


In [17]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df["id_code"].values, train_df["diagnosis"].values))
train_ds = train_ds.map(lambda x, y: load_image(x, y, augment=True), num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [18]:
val_ds = tf.data.Dataset.from_tensor_slices((val_df["id_code"].values, val_df["diagnosis"].values))
val_ds = val_ds.map(lambda x, y: load_image(x, y, augment=False), num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)



In [19]:
#class imbalance
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_df['diagnosis']),
                                     y=train_df['diagnosis'])
class_weights = dict(enumerate(class_weights))


In [20]:
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base layers initially

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(5, activation='softmax')  # 5 classes for APTOS
])

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [21]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,184 (16.08 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    class_weight=class_weights,
                    verbose=1)
